In [6]:
# get the json file from the folder
list = !ls finalJson

In [7]:
list

['nativeFemale1.json',
 'nativeFemale2.json',
 'nativeFemale3.json',
 'nativeFemale4.json',
 'nativeFemale5.json',
 'nativeMale1.json',
 'nativeMale2.json',
 'nativeMale3.json',
 'nativeMale4.json',
 'nativeMale5.json',
 'nonNativeFemale1.json',
 'nonNativeFemale2.json',
 'nonNativeFemale3.json',
 'nonNativeFemale4.json',
 'nonNativeFemale5.json',
 'nonNativeMale1.json',
 'nonNativeMale2.json',
 'nonNativeMale3.json',
 'nonNativeMale4.json',
 'nonNativeMale5.json']

In [5]:
# loop through all files to get transcriptions
for filename in list:
    !aws transcribe start-transcription-job \
        --cli-input-json file://finalJson/{filename}

{
    "TranscriptionJob": {
        "TranscriptionJobName": "nativeFemale1",
        "TranscriptionJobStatus": "IN_PROGRESS",
        "LanguageCode": "en-US",
        "Media": {
            "MediaFileUri": "s3://final-resources/finalRecordings/nativeFemale1.mp3"
        },
        "StartTime": 1637964975.317,
        "CreationTime": 1637964975.28
    }
}
{
    "TranscriptionJob": {
        "TranscriptionJobName": "nativeFemale2",
        "TranscriptionJobStatus": "IN_PROGRESS",
        "LanguageCode": "en-US",
        "Media": {
            "MediaFileUri": "s3://final-resources/finalRecordings/nativeFemale2.mp3"
        },
        "StartTime": 1637964976.274,
        "CreationTime": 1637964976.249
    }
}
{
    "TranscriptionJob": {
        "TranscriptionJobName": "nativeFemale3",
        "TranscriptionJobStatus": "IN_PROGRESS",
        "LanguageCode": "en-US",
        "Media": {
            "MediaFileUri": "s3://final-resources/finalRecordings/nativeFemale3.mp3"
        },
        "St

In [8]:
for filename in list:
    !aws s3 cp s3://final-resources/{filename} {filename}

download: s3://final-resources/nativeFemale1.json to ./nativeFemale1.json
download: s3://final-resources/nativeFemale2.json to ./nativeFemale2.json
download: s3://final-resources/nativeFemale3.json to ./nativeFemale3.json
download: s3://final-resources/nativeFemale4.json to ./nativeFemale4.json
download: s3://final-resources/nativeFemale5.json to ./nativeFemale5.json
download: s3://final-resources/nativeMale1.json to ./nativeMale1.json
download: s3://final-resources/nativeMale2.json to ./nativeMale2.json
download: s3://final-resources/nativeMale3.json to ./nativeMale3.json
download: s3://final-resources/nativeMale4.json to ./nativeMale4.json
download: s3://final-resources/nativeMale5.json to ./nativeMale5.json
download: s3://final-resources/nonNativeFemale1.json to ./nonNativeFemale1.json
download: s3://final-resources/nonNativeFemale2.json to ./nonNativeFemale2.json
download: s3://final-resources/nonNativeFemale3.json to ./nonNativeFemale3.json
download: s3://final-resources/nonNative

In [9]:
import pandas as pd
import json

In [16]:
# write the files into texts and store them
for filename in list:
    with open(filename, "r") as f:
        array = json.load(f)
    textfile = filename[0:len(filename)-5]+".txt"
    with open("finalTranscriptions/"+textfile, "w") as t:
        t.write(array["results"]["transcripts"][0]["transcript"])

In [30]:
# record the confidence into csv files and store them
for filename in list:
    with open(filename, "r") as f:
        array = json.load(f)
    dict = {}
    index = 0
    csvfile = filename[0:len(filename)-5]+".csv"
    for i in range(len(array['results']['items'])):
        if array['results']['items'][i]['type'] != 'punctuation':
            index += 1
            dict[index] = [array['results']['items'][i]['alternatives'][0]['confidence'], array['results']['items'][i]['alternatives'][0]['content']]
    DF = pd.DataFrame.from_dict(dict, orient='index', columns = ["confidence", "content"])
    DF.to_csv('finalConfidence/'+csvfile, encoding='utf-8', index=False)

In [36]:
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 13.9 MB 23.1 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [47]:
import re

In [48]:
# open the sample text file
with open("AlexaSample.txt", "r") as f:
    sampleText = f.read()
sampleText = re.sub(r'[^\w\s]','',sampleText)

In [55]:
# create sample dictionary 
# we are assuming that aws transcription only have minor mistakes in transcribing texts
sample = nlp(sampleText)
sampleToken = [token.lemma_.lower() for token in sample]
sampledict={}
for word in sampleToken:
    if word in sampledict:
        sampledict[word] += 1
    else:
        sampledict[word] = 1

In [41]:
textlist = !ls finalTranscriptions

In [58]:
result = {}
for filename in textlist:
    with open('finalTranscriptions/'+filename, 'r') as f:
        inputText = f.read()
    inputText = re.sub(r'[^\w\s]','',inputText)
    doc = nlp(inputText)
    docToken = [token.lemma_.lower() for token in doc]
    dict = {}
    for word in docToken:
        if word in dict:
            dict[word] += 1
        else:
            dict[word] = 1
    corrCount = 0
    for key in sampledict:
        if key in dict:
            if sampledict[key] == dict[key]:
                corrCount += sampledict[key]
    result[filename[0:len(filename)-4]] = corrCount/len(sampleToken)

{'nativeFemale1': 0.922077922077922,
 'nativeFemale2': 0.948051948051948,
 'nativeFemale3': 0.7402597402597403,
 'nativeFemale4': 0.987012987012987,
 'nativeFemale5': 0.922077922077922,
 'nativeMale1': 0.987012987012987,
 'nativeMale2': 0.987012987012987,
 'nativeMale3': 0.987012987012987,
 'nativeMale4': 0.987012987012987,
 'nativeMale5': 0.987012987012987,
 'nonNativeFemale1': 0.6363636363636364,
 'nonNativeFemale2': 0.8961038961038961,
 'nonNativeFemale3': 0.8831168831168831,
 'nonNativeFemale4': 0.922077922077922,
 'nonNativeFemale5': 0.8701298701298701,
 'nonNativeMale1': 0.948051948051948,
 'nonNativeMale2': 0.961038961038961,
 'nonNativeMale3': 0.7792207792207793,
 'nonNativeMale4': 0.974025974025974,
 'nonNativeMale5': 0.987012987012987}

In [83]:
# 1 represents true, 0 represents false
dfdict = {}
for key in result:
    if key[0:6] == 'native':
        Native = 1
        if key[6:10] == 'Male':
            Female = 0
        else:
            Female = 1
    else:
        Native = 0
        if key[9:13] == 'Male':
            Female = 0
        else:
            Female = 1
    dfdict[key] = [result[key], Native, Female]

In [85]:
df = pd.DataFrame.from_dict(dfdict, orient='index', columns = ["Accuracy","Native", "Female"])
df.to_csv('Accuracy.csv', encoding='utf-8', index=False)